## ECoG+STN-LFP analysis: demo 

In [2]:
# Import libraries
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score
import mne
from mne.decoding import SPoC
mne.set_log_level(verbose='warning') #to avoid info at terminal
import pickle 
import sys
# from Utilities folder
sys.path.insert(1, './Utilities/icn_m1')
import os
sys.path.insert(1, './Utilities/')
from FilterBank import FilterBank
from ML_models import get_model

from collections import OrderedDict
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
# define settings
# change them accordinly
settings = {}
settings['data_path'] = "C:/Users/Pilin/Dropbox (Brain Modulation Lab)/Experiments/CRCNS/Data/Epoched/"
settings['num_patients'] = ['000'] # for this example we only use one subject
# subfolders indicates the session in this dataset
settings['subfolders']=[['ses-right']]  # this subject only has one session

In [ ]:
# define some experiments and model parameters
laterality = ["CON", "IPS"]
signal = ["STN", "ECOG"]

cv = KFold(n_splits=5, shuffle=False)
spoc = SPoC(n_components=1, log=True, reg='oas', transform_into ='average_power', rank='full')
USED_MODEL = 3 # 3 == GLM with alpha 0.5

### Load the data from both modalities.
This data was already pre-processed, following these steps:
1. Epochs of 100 ms were extracted.
2. Band-passed filtered epoched data at 8 frequency bands ([4, 8], [8, 12], [13, 20], [20, 35], [13, 35], [60, 80], [90, 200], [60, 200])
3. The target variable was downsampled accordinly to the 100 ms epoch lenght.

If you want to run this code with your own data, please be sure of arranging your data in a 4d array as follows:
(epochs, channels, samples, frequency bands)

In [ ]:
# declare saving variable
Ypre_tr = OrderedDict()
score_tr = OrderedDict()
Ypre_te = OrderedDict()
score_te = OrderedDict()
Patterns_ecog = OrderedDict()
Filters_ecog = OrderedDict()
Patterns_stn = OrderedDict()
Filters_stn = OrderedDict()
Coef = OrderedDict()
hyperparams = OrderedDict()
Label_tr = OrderedDict()
Label_te = OrderedDict()          

In [ ]:
# get data
s = 0 # when working with all subjects, this is a for
subfolders=settings["subfolders"][s]
ss = 0 # when working with all subjects, this is a for, since there are subjects which have more than one session.
X_ECOG = [] # to append data
X_STN =[] 
Y_con = []
Y_ips = []
list_of_files_ecog = os.listdir(settings['data_path']+'ECOG') # list of files in the current directory
list_of_files_stn = os.listdir(settings['data_path']+'STN') 

file_name_ = 'ECOG_epochs_sub_' + settings['num_patients'][s] + '_sess_'+subfolders[ss][4:]

file_ecog = [each_file for each_file in list_of_files_ecog if each_file.startswith(file_name_)]
file_name_='STN_epochs_sub_' + settings['num_patients'][s] + '_sess_'+subfolders[ss][4:]

# only load data from runs in which both modali
file_stn= [each_file for each_file in list_of_files_stn if each_file.startswith(file_name_)]
idx_file = [f for f in file_stn if list(set() & set(file_ecog))]
matching_stn = [f for f in file_stn if any(f[4:] in xs for xs in file_ecog)]
matching_ecog = [f for f in file_ecog if any(f[4:] in xs for xs in file_stn)]

if len(matching_ecog) != len(matching_stn):
    raise('Error loading data')

for e in range(len(matching_ecog)):
    with open(settings['data_path'] +'ECOG/' + matching_ecog[e], 'rb') as handle:
        sub_ = pickle.load(handle)    
        data = sub_['epochs']
        X_ECOG.append(data)
        label_ips = sub_['label_ips']
        label_con = sub_['label_con']
        Y_con.append(label_con)
        Y_ips.append(label_ips)
    with open(settings['data_path'] +'STN/' + matching_stn[e], 'rb') as handle:
        sub_ = pickle.load(handle)
        data = sub_['epochs']
        X_STN.append(data)           

X_ECOG = np.concatenate(X_ECOG, axis=0)
X_STN = np.concatenate(X_STN, axis=0)
Y_con = np.concatenate(Y_con, axis=0)
Y_ips = np.concatenate(Y_ips, axis=0)  

In [ ]:
for l, mov in enumerate(laterality):
    print("training %s" %mov)
    score_tr[mov] = []
    score_te[mov] = []
    Ypre_tr[mov] = []
    Ypre_te[mov] = []
    Label_tr[mov] = []
    Label_te[mov] = []
    Patterns_ecog[mov] = []
    Filters_ecog[mov] = []
    Patterns_stn[mov] = []
    Filters_stn[mov] = []
    Coef[mov] = []
    hyperparams[mov] = []
    if l==0:
        label=Y_con
    else:
        label=Y_ips

    features_ecog=FilterBank(estimator=spoc)
    features_stn=FilterBank(estimator=spoc)

    X_ECOG=X_ECOG.astype('float64')
    X_STN=X_STN.astype('float64')

    gtr=[]
    gte=[]
    for train_index, test_index in cv.split(label):
        Ztr, Zte=label[train_index], label[test_index]
        Xtr_ecog, Xte_ecog=X_ECOG[train_index], X_ECOG[test_index]
        Xtr_stn, Xte_stn=X_STN[train_index], X_STN[test_index]
        # features are learned from each modality and then concatenated
        Gtr_ecog_cspoc, Gtr_stn_cspoc = features_ecog.fit_transform(Xtr_ecog, Ztr), features_stn.fit_transform(Xtr_stn, Ztr)
        Gte_ecog_cspoc, Gte_stn_cspoc = features_ecog.transform(Xte_ecog), features_stn.transform(Xte_stn)

        dat_tr=np.hstack((Gtr_ecog_cspoc,Gtr_stn_cspoc))
        dat_te=np.hstack((Gte_ecog_cspoc,Gte_stn_cspoc))

        Label_te[mov].append(Zte)
        Label_tr[mov].append(Ztr)
        
        # get the model
        clf, optimizer = get_model(USED_MODEL, x=dat_tr, y=Ztr)

        scaler = StandardScaler()
        scaler.fit(dat_tr)
        dat_tr = scaler.transform(dat_tr)
        dat_te = scaler.transform(dat_te)
        
        # fit the model
        clf.fit(dat_tr, Ztr)
        Ypre_te[mov].append(clf.predict(dat_te))
        Ypre_tr[mov].append(clf.predict(dat_tr))
        
        # predict
        r2_te = r2_score(Zte, clf.predict(dat_te))
        if r2_te < 0: r2_te = 0
        score_te[mov].append(r2_te)
        r2_tr = r2_score(Ztr,clf.predict(dat_tr))
        if r2_tr < 0: r2_tr = 0
        score_tr[mov].append(r2_tr)
        
        # save model info
        Filters_ecog[mov].append(features_ecog.filters)
        Patterns_ecog[mov].append(features_ecog.patterns)

        Filters_stn[mov].append(features_stn.filters)
        Patterns_stn[mov].append(features_stn.patterns)

        if USED_MODEL > 1:
            Coef[mov].append(clf.beta_)
        else:
            Coef[mov].append(clf.coef_)

        hyperparams[mov].append(optimizer['params'])
        del Xtr_ecog, Xte_ecog, Xtr_stn, Xte_stn